# Proyecto Data Engineering – Consultas Analíticas 

Autor: Gian Paolo Robelo, Juan Diego Letona, Andrea Soto
Base: Lakehouse DuckDB  
Ruta: `data/lakehouse.duckdb`  

---

## Aggregated Gold

### 1. Top 10 artículos más populares en (idioma) para el año (año)
```sql
SELECT rn, curr_title, clicks_year
FROM gs_top10_by_year
WHERE lang = 'es' AND year = 2024
ORDER BY rn;
```

#### **Tiempo de Ejecución**: 0.659 segundos
#### **Explicación**: Muestra los 10 artículos con más clics en un idioma y año específicos. La partición lang/year/month es clave aquí porque el filtro lang='es' AND year=2024 permite que DuckDB solo lea los Parquet de esa combinación, evitando escanear todo el dataset



### 1.1 Top 10 artículos más populares en español para los años (años)
```sql
SELECT year, rn, curr_title, clicks_year
FROM gs_top10_by_year
WHERE lang = 'es'
  AND year IN (2024, 2025)
ORDER BY year, rn;
```

#### **Tiempo de Ejecución**: 0.824 segundos
#### **Explicación**: Compara los rankings de distintos años en el mismo idioma. Gracias a la partición, DuckDB solo accede a las carpetas lang=es/year=2024 y lang=es/year=2025, acelerando el análisis multianual.


### 2. Top 10 artículos con más clics de todos los tiempos en español
```sql
SELECT curr_title AS title, clicks_all_time AS clicks
FROM gs_all_time_top
WHERE lang = 'es'
ORDER BY clicks_all_time DESC
LIMIT 10;
```
#### **Tiempo de Ejecución**: 1.517 segundos
#### **Explicación**: Consulta agregada que calcula los artículos más visitados históricamente. El uso de lang en la partición evita escanear todos los idiomas, reduciendo la lectura solo a los Parquet del español.

### 3. Comparar el top 10 de artículos más populares entre idiomas (idiomas) para el año (año)
```sql
SELECT lang, rn, curr_title, clicks_year
FROM gs_top10_by_year
WHERE year = 2024
  AND lang IN ('es','en','pt')
ORDER BY lang, rn;
```
#### **Tiempo de Ejecución**: 0.489 segundos
#### **Explicación**: Compara tendencias entre idiomas. La partición permite que DuckDB busque solo los directorios de esos idiomas, evitando I/O innecesario de otros

### 4. Artículos nuevos en el top 10 (año actual vs anterior)
```sql
WITH y_cur AS (
  SELECT curr_title FROM gs_top10_by_year WHERE lang = 'es' AND year = 2020
),
y_prev AS (
  SELECT curr_title FROM gs_top10_by_year WHERE lang = 'es' AND year = 2019
)
SELECT c.curr_title AS new_in_top10_2024
FROM y_cur c
LEFT JOIN y_prev p ON p.curr_title = c.curr_title
WHERE p.curr_title IS NULL
ORDER BY 1;
```
#### **Tiempo de Ejecución**: 0.916 segundos
#### **Explicación**: Detecta nuevos artículos en el top 10 comparando dos años consecutivos. Solo se leen las carpetas de year=2020 y year=2019, lo que minimiza el escaneo de datos históricos.

### 5. Artículos con mayor crecimiento mensual en español
```sql
WITH lastp AS (
  SELECT MAX(make_date(year, month, 1)) AS p
  FROM gs_trending_mom
  WHERE lang = 'es'
)
SELECT year, month, curr_title, clicks_prev, clicks_now, abs_delta, pct_delta
FROM gs_trending_mom
WHERE lang = 'es'
  AND make_date(year, month, 1) = (SELECT p FROM lastp)
ORDER BY pct_delta DESC NULLS LAST, abs_delta DESC
LIMIT 20;
```
#### **Tiempo de Ejecución**: 4.930 segundos
#### **Explicación**: Encuentra los artículos con mayor incremento de clics entre meses. La estructura lang/year/month facilita localizar rápidamente el último mes disponible sin leer todo el histórico

### 6. Artículos que más años han estado en el top 10 (idioma)
```sql
SELECT curr_title, COUNT(*) AS years_in_top10
FROM gs_top10_by_year
WHERE lang = 'en'
GROUP BY curr_title
ORDER BY years_in_top10 DESC, curr_title
LIMIT 20;
```
#### **Tiempo de Ejecución**: 0.613 segundos
#### **Explicación**: Cuenta los años que cada artículo ha sido parte del top 10. La partición por lang reduce la búsqueda a un idioma, lo cual optimiza el conteo y agrupación por año

### 7. Artículos populares en ambos idiomas (español e inglés)
```sql
WITH es AS (SELECT curr_title, clicks_year FROM gs_top10_by_year WHERE lang='es' AND year=2024),
     en AS (SELECT curr_title, clicks_year FROM gs_top10_by_year WHERE lang='en' AND year=2024)
SELECT es.curr_title, es.clicks_year AS clicks_es, en.clicks_year AS clicks_en
FROM es JOIN en ON en.curr_title = es.curr_title
ORDER BY clicks_es + clicks_en DESC
LIMIT 20;
```
#### **Tiempo de Ejecución**: 0.781 segundos
#### **Explicación**: Identifica artículos comunes entre idiomas. Cada subconsulta lee solo su partición (lang=es y lang=en), optimizando el JOIN entre idiomas.

### 8. Artículo más popular por año (idioma)
```sql
SELECT year, curr_title, clicks_year
FROM gs_top10_by_year
WHERE lang = 'es' AND rn = 1
ORDER BY year DESC;
```
#### **Tiempo de Ejecución**: 0.694 segundos
#### **Explicación**: Obtiene el líder anual por idioma. Gracias al filtro lang='es', solo se consultan los directorios del español, sin necesidad de revisar todos los idiomas

### 9. Artículos que salieron del top 10 y luego regresaron (idioma)
```sql
WITH t AS (
  SELECT year, curr_title, rn
  FROM gs_top10_by_year
  WHERE lang='en' AND rn<=10
),
r AS (
  SELECT curr_title, year,
         LAG(year) OVER (PARTITION BY curr_title ORDER BY year) AS prev_year
  FROM t
)
SELECT curr_title, prev_year AS last_seen_year, year AS comeback_year
FROM r
WHERE prev_year IS NOT NULL AND year - prev_year > 1
ORDER BY comeback_year DESC, curr_title;
```
#### **Tiempo de Ejecución**: 0.755 segundos
#### **Explicación**: Analiza la persistencia de artículos en el ranking. La segmentación por lang hace que DuckDB solo lea el historial del idioma seleccionado, mejorando rendimiento.

### 10. Margen de clics entre líder y subcampeón (idioma, año)
```sql
WITH params AS (SELECT 'es'::VARCHAR AS lang, 2024::INT AS year),
t AS (
  SELECT rn, curr_title, clicks_year
  FROM gs_top10_by_year
  WHERE lang=(SELECT lang FROM params) AND year=(SELECT year FROM params) AND rn<=2
)
SELECT
  MAX(CASE WHEN rn=1 THEN curr_title END) AS leader,
  MAX(CASE WHEN rn=1 THEN clicks_year END) AS leader_clicks,
  MAX(CASE WHEN rn=2 THEN curr_title END) AS runner_up,
  MAX(CASE WHEN rn=2 THEN clicks_year END) AS runner_up_clicks,
  (MAX(CASE WHEN rn=1 THEN clicks_year END) - MAX(CASE WHEN rn=2 THEN clicks_year END)) AS margin
FROM t;
```
#### **Tiempo de Ejecución**: 1.176 segundos
#### **Explicación**: Evalúa la diferencia de clics entre los primeros puestos. La partición lang/year acota la lectura a un solo conjunto anual del idioma elegido.